In [19]:
import pandas as pd
df = pd.read_csv('rdata.csv')
df.head()
df.columns

df = df[[
    'TIMESTAMP',
    'FEEDCODE_HSI',
    'FEEDCODE_HHI',
    'LAST_BID_PRICE_HSI',
    'LAST_ASK_PRICE_HSI',
    'LAST_BID_VOLUME_HSI',
    'LAST_ASK_VOLUME_HSI',
    'LAST_BID_PRICE_HHI',
    'LAST_ASK_PRICE_HHI',
    'LAST_BID_VOLUME_HHI',
    'LAST_ASK_VOLUME_HHI'
]]

df

,TIMESTAMP,FEEDCODE_HSI,FEEDCODE_HHI,LAST_BID_PRICE_HSI,LAST_ASK_PRICE_HSI,LAST_BID_VOLUME_HSI,LAST_ASK_VOLUME_HSI,LAST_BID_PRICE_HHI,LAST_ASK_PRICE_HHI,LAST_BID_VOLUME_HHI,LAST_ASK_VOLUME_HHI
0,2021-01-04 01:17:00+00:00,HSIF1,HHIF1,27145.0,27147.0,3.0,2.0,10660.0,10662.0,3.0,3.0
1,2021-01-04 01:18:00+00:00,HSIF1,HHIF1,27140.0,27143.0,2.0,2.0,10658.0,10660.0,6.0,3.0
2,2021-01-04 01:19:00+00:00,HSIF1,HHIF1,27132.0,27135.0,3.0,4.0,10654.0,10655.0,1.0,2.0
3,2021-01-04 01:20:00+00:00,HSIF1,HHIF1,27126.0,27128.0,1.0,3.0,10658.0,10660.0,12.0,2.0
4,2021-01-04 01:21:00+00:00,HSIF1,HHIF1,27127.0,27130.0,2.0,2.0,10660.0,10662.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
682716,2023-12-29 18:56:00+00:00,HSIF4,HHIF4,17156.0,17159.0,2.0,1.0,5814.0,5815.0,13.0,53.0
682717,2023-12-29 18:57:00+00:00,HSIF4,HHIF4,17157.0,17160.0,1.0,1.0,5814.0,5815.0,13.0,65.0
682718,2023-12-29 18:58:00+00:00,HSIF4,HHIF4,17158.0,17160.0,4.0,1.0,5815.0,5816.0,12.0,13.0
682719,2023-12-29 18:59:00+00:00,HSIF4,HHIF4,17153.0,17156.0,5.0,1.0,5814.0,5816.0,15.0,9.0


In [20]:
import numpy as np

# Calculate midpoint prices
df['MIDPOINT_HSI'] = (df['LAST_BID_PRICE_HSI'] + df['LAST_ASK_PRICE_HSI']) / 2
df['MIDPOINT_HHI'] = (df['LAST_BID_PRICE_HHI'] + df['LAST_ASK_PRICE_HHI']) / 2

# Drop NaN values caused by the shift
df = df.dropna()

In [3]:
from sklearn.linear_model import LinearRegression

# Linear regression
X = df['MIDPOINT_HHI'].values.reshape(-1, 1)  # Predictor
y = df['MIDPOINT_HSI'].values  # Target
model = LinearRegression().fit(X, y)

# Make a deep copy of the DataFrame to avoid issues with SettingWithCopyWarning
df = df.copy()

# Predict and calculate residuals
df['PREDICTED_HSI'] = model.predict(X)
df['RESIDUALS'] = df['MIDPOINT_HSI'] - df['PREDICTED_HSI']

In [4]:
# Ensure df is a standalone DataFrame
df = df.copy()

# Rolling mean and standard deviation for residuals
df['RESIDUAL_MEAN'] = df['RESIDUALS'].rolling(window=50).mean()
df['RESIDUAL_STD'] = df['RESIDUALS'].rolling(window=50).std()
df['Z_SCORE'] = (df['RESIDUALS'] - df['RESIDUAL_MEAN']) / df['RESIDUAL_STD']

In [5]:
df.dropna()

,TIMESTAMP,FEEDCODE_HSI,FEEDCODE_HHI,LAST_BID_PRICE_HSI,LAST_ASK_PRICE_HSI,LAST_BID_VOLUME_HSI,LAST_ASK_VOLUME_HSI,LAST_BID_PRICE_HHI,LAST_ASK_PRICE_HHI,LAST_BID_VOLUME_HHI,LAST_ASK_VOLUME_HHI,MIDPOINT_HSI,MIDPOINT_HHI,PREDICTED_HSI,RESIDUALS,RESIDUAL_MEAN,RESIDUAL_STD,Z_SCORE
49,2021-01-04 02:06:00+00:00,HSIF1,HHIF1,27412.0,27414.0,1.0,2.0,10711.0,10712.0,4.0,8.0,27413.0,10711.5,28540.113180,-1127.113180,-1221.052600,56.450775,1.664094
50,2021-01-04 02:07:00+00:00,HSIF1,HHIF1,27408.0,27409.0,2.0,1.0,10710.0,10712.0,9.0,10.0,27408.5,10711.0,28538.998082,-1130.498082,-1218.032797,57.185609,1.530712
51,2021-01-04 02:08:00+00:00,HSIF1,HHIF1,27395.0,27397.0,3.0,4.0,10707.0,10708.0,7.0,2.0,27396.0,10707.5,28531.192395,-1135.192395,-1215.106087,57.612734,1.387084
52,2021-01-04 02:09:00+00:00,HSIF1,HHIF1,27411.0,27412.0,1.0,1.0,10713.0,10715.0,11.0,8.0,27411.5,10714.0,28545.688671,-1134.188671,-1212.200020,57.962329,1.345897
53,2021-01-04 02:10:00+00:00,HSIF1,HHIF1,27401.0,27403.0,6.0,1.0,10706.0,10708.0,12.0,7.0,27402.0,10707.0,28530.077297,-1128.077297,-1208.841009,57.871660,1.395566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682716,2023-12-29 18:56:00+00:00,HSIF4,HHIF4,17156.0,17159.0,2.0,1.0,5814.0,5815.0,13.0,53.0,17157.5,5814.5,17618.842233,-461.342233,-462.868876,1.131197,1.349582
682717,2023-12-29 18:57:00+00:00,HSIF4,HHIF4,17157.0,17160.0,1.0,1.0,5814.0,5815.0,13.0,65.0,17158.5,5814.5,17618.842233,-460.342233,-462.821895,1.186168,2.090482
682718,2023-12-29 18:58:00+00:00,HSIF4,HHIF4,17158.0,17160.0,4.0,1.0,5815.0,5816.0,12.0,13.0,17159.0,5815.5,17621.072429,-462.072429,-462.784123,1.179202,0.603538
682719,2023-12-29 18:59:00+00:00,HSIF4,HHIF4,17153.0,17156.0,5.0,1.0,5814.0,5816.0,15.0,9.0,17154.5,5815.0,17619.957331,-465.457331,-462.844048,1.237159,-2.112327


In [6]:
import pandas as pd

# Initialize previous signals for HSI and HHI
prev_hsi_signal = 0
prev_hhi_signal = 0

# Define the signal generation function
def generate_signals(df, entry_threshold=2, exit_threshold=0.5):
    """
    Generate HSI and HHI signals based on Z-Score for the entire DataFrame.
    """
    # Initialize signal columns
    df['HSI_SIGNAL'] = 0
    df['HHI_SIGNAL'] = 0

    # Loop through rows using efficient iteration with .iloc
    prev_hsi_signal = 0
    prev_hhi_signal = 0

    for i in range(len(df)):
        z_score = df.at[i, 'Z_SCORE']
        hsi_signal = 0
        hhi_signal = 0

        # Entry signals: Open new positions
        if z_score < -entry_threshold:  # Long Spread: Buy HSI, Sell HHI
            hsi_signal = +1  # Buy HSI
            hhi_signal = -1  # Sell HHI
        elif z_score > entry_threshold:  # Short Spread: Sell HSI, Buy HHI
            hsi_signal = -1  # Sell HSI
            hhi_signal = +1  # Buy HHI

        # Exit signals: Close existing positions
        elif abs(z_score) < exit_threshold:
            # Close HSI position
            if prev_hsi_signal == +1:  # Previously long HSI
                hsi_signal = -1  # Sell HSI
            elif prev_hsi_signal == -1:  # Previously short HSI
                hsi_signal = +1  # Buy HSI

            # Close HHI position
            if prev_hhi_signal == +1:  # Previously long HHI
                hhi_signal = -1  # Sell HHI
            elif prev_hhi_signal == -1:  # Previously short HHI
                hhi_signal = +1  # Buy HHI

        # Hold condition: Maintain current position if no action is taken
        if hsi_signal == 0 and hhi_signal == 0:
            hsi_signal = prev_hsi_signal
            hhi_signal = prev_hhi_signal

        # Update the DataFrame with the calculated signals
        df.at[i, 'HSI_SIGNAL'] = hsi_signal
        df.at[i, 'HHI_SIGNAL'] = hhi_signal

        # Update previous signals
        prev_hsi_signal = hsi_signal
        prev_hhi_signal = hhi_signal

    return df

# Apply the signal generation function
df = generate_signals(df)

In [7]:
import numpy as np

# Generate trades for HSI
hsi_trades = df[df['HSI_SIGNAL'] != 0].copy()
hsi_trades['FEEDCODE'] = hsi_trades['FEEDCODE_HSI']
hsi_trades['BUYSELL'] = np.where(hsi_trades['HSI_SIGNAL'] == 1, 'B', 'S')
hsi_trades['PRICE'] = np.where(
    hsi_trades['HSI_SIGNAL'] == 1, hsi_trades['LAST_ASK_PRICE_HSI'], hsi_trades['LAST_BID_PRICE_HSI']
)
hsi_trades['VOLUME'] = np.where(
    hsi_trades['HSI_SIGNAL'] == 1, hsi_trades['LAST_ASK_VOLUME_HSI'], hsi_trades['LAST_BID_VOLUME_HSI']
)
hsi_trades = hsi_trades[['TIMESTAMP', 'FEEDCODE', 'BUYSELL', 'PRICE', 'VOLUME']]

# Generate trades for HHI
hhi_trades = df[df['HHI_SIGNAL'] != 0].copy()
hhi_trades['FEEDCODE'] = hhi_trades['FEEDCODE_HHI']
hhi_trades['BUYSELL'] = np.where(hhi_trades['HHI_SIGNAL'] == 1, 'B', 'S')
hhi_trades['PRICE'] = np.where(
    hhi_trades['HHI_SIGNAL'] == 1, hhi_trades['LAST_ASK_PRICE_HHI'], hhi_trades['LAST_BID_PRICE_HHI']
)
hhi_trades['VOLUME'] = np.where(
    hhi_trades['HHI_SIGNAL'] == 1, hhi_trades['LAST_ASK_VOLUME_HHI'], hhi_trades['LAST_BID_VOLUME_HHI']
)
hhi_trades = hhi_trades[['TIMESTAMP', 'FEEDCODE', 'BUYSELL', 'PRICE', 'VOLUME']]

# Combine HSI and HHI trades
trade_table = pd.concat([hsi_trades, hhi_trades], ignore_index=True)

# Handle aggregation for same TIMESTAMP and FEEDCODE
trade_table['PRICE_VOLUME'] = trade_table['PRICE'] * trade_table['VOLUME']
aggregated_table = (
    trade_table.groupby(['TIMESTAMP', 'FEEDCODE', 'BUYSELL'], as_index=False)
    .agg(
        VOLUME=('VOLUME', 'sum'),
        PRICE_VOLUME=('PRICE_VOLUME', 'sum')
    )
)
aggregated_table['PRICE'] = aggregated_table['PRICE_VOLUME'] / aggregated_table['VOLUME']

# Drop intermediate calculation column
trades = aggregated_table[['TIMESTAMP', 'FEEDCODE', 'BUYSELL', 'PRICE', 'VOLUME']]


In [8]:
trades

,TIMESTAMP,FEEDCODE,BUYSELL,PRICE,VOLUME
0,2021-01-04 03:02:00+00:00,HHIF1,S,10672.0,15.0
1,2021-01-04 03:02:00+00:00,HSIF1,B,27342.0,2.0
2,2021-01-04 03:03:00+00:00,HHIF1,S,10673.0,8.0
3,2021-01-04 03:03:00+00:00,HSIF1,B,27348.0,2.0
4,2021-01-04 03:04:00+00:00,HHIF1,S,10671.0,12.0
...,...,...,...,...,...
1365227,2023-12-29 18:58:00+00:00,HSIF4,S,17158.0,4.0
1365228,2023-12-29 18:59:00+00:00,HHIF4,S,5814.0,15.0
1365229,2023-12-29 18:59:00+00:00,HSIF4,B,17156.0,1.0
1365230,2023-12-29 19:00:00+00:00,HHIF4,S,5814.0,3.0


In [9]:
trades.to_csv("tradies.csv")

In [13]:
trades = pd.read_csv("tradies.csv")

In [14]:
trades

,Unnamed: 0,TIMESTAMP,FEEDCODE,BUYSELL,PRICE,VOLUME
0,0,2021-01-04 03:02:00+00:00,HHIF1,S,10672.0,15.0
1,1,2021-01-04 03:02:00+00:00,HSIF1,B,27342.0,2.0
2,2,2021-01-04 03:03:00+00:00,HHIF1,S,10673.0,8.0
3,3,2021-01-04 03:03:00+00:00,HSIF1,B,27348.0,2.0
4,4,2021-01-04 03:04:00+00:00,HHIF1,S,10671.0,12.0
...,...,...,...,...,...,...
1365227,1365227,2023-12-29 18:58:00+00:00,HSIF4,S,17158.0,4.0
1365228,1365228,2023-12-29 18:59:00+00:00,HHIF4,S,5814.0,15.0
1365229,1365229,2023-12-29 18:59:00+00:00,HSIF4,B,17156.0,1.0
1365230,1365230,2023-12-29 19:00:00+00:00,HHIF4,S,5814.0,3.0


In [15]:
trades = trades[~trades.index.duplicated(keep='first')]
trades

,Unnamed: 0,TIMESTAMP,FEEDCODE,BUYSELL,PRICE,VOLUME
0,0,2021-01-04 03:02:00+00:00,HHIF1,S,10672.0,15.0
1,1,2021-01-04 03:02:00+00:00,HSIF1,B,27342.0,2.0
2,2,2021-01-04 03:03:00+00:00,HHIF1,S,10673.0,8.0
3,3,2021-01-04 03:03:00+00:00,HSIF1,B,27348.0,2.0
4,4,2021-01-04 03:04:00+00:00,HHIF1,S,10671.0,12.0
...,...,...,...,...,...,...
1365227,1365227,2023-12-29 18:58:00+00:00,HSIF4,S,17158.0,4.0
1365228,1365228,2023-12-29 18:59:00+00:00,HHIF4,S,5814.0,15.0
1365229,1365229,2023-12-29 18:59:00+00:00,HSIF4,B,17156.0,1.0
1365230,1365230,2023-12-29 19:00:00+00:00,HHIF4,S,5814.0,3.0


In [16]:
trades['TIMESTAMP'] = pd.to_datetime(trades['TIMESTAMP']).dt.tz_convert('UTC')

In [18]:
trades.to_csv('tradies.csv')